---
## 5. Enviar Emails REALS amb Gmail API

Ara ve la part emocionant: **enviarem emails REALS** amb els RFQs generats!

### 5.1 Configuració Gmail API (Pas a Pas)

Per poder enviar i rebre emails, necessitem configurar Gmail API:

#### 📋 Passos de configuració (fer ABANS de la formació):

**1. Instal·lar llibreries necessàries:**

```bash
pip install --upgrade google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client
```

**2. Crear projecte a Google Cloud Console:**

- Anar a: https://console.cloud.google.com/
- Crear un projecte nou: "CrewAI Gmail Integration"
- Seleccionar el projecte

**3. Activar Gmail API:**

- Al menú lateral: "APIs & Services" → "Library"
- Cercar "Gmail API"
- Clicar "Enable"

**4. Crear credencials OAuth 2.0:**

- "APIs & Services" → "Credentials"
- "Create Credentials" → "OAuth client ID"
- Application type: **"Desktop app"**
- Name: "CrewAI Desktop Client"
- Descarregar el JSON → guardar com `credentials.json`

**5. Col·locar credentials.json:**

```
project/
├── credentials.json  ← Aquí!
├── data/
└── notebooks/
```

**6. Primera autenticació:**

La primera vegada que executis el codi:
- S'obrirà un navegador
- Inicia sessió amb el teu compte Gmail
- Accepta els permisos
- Es crearà `token.json` (guarda't-ho!)

**⚠️ Seguretat:**
- NO pugis `credentials.json` ni `token.json` a GitHub
- Afegeix-los al `.gitignore`

### 5.2 Instal·lar dependències

Executem això primer per assegurar que tenim tot:

In [ ]:
# Instal·lar llibreries Gmail API
!pip install -q --upgrade google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

print("✅ Llibreries Gmail API instal·lades!")

### 5.3 Tool: Authenticate Gmail

Primer creem una tool per autenticar amb Gmail:

In [ ]:
import os
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Permisos necessaris
SCOPES = [
    'https://www.googleapis.com/auth/gmail.send',      # Enviar emails
    'https://www.googleapis.com/auth/gmail.readonly',  # Llegir emails
    'https://www.googleapis.com/auth/gmail.modify'     # Marcar com llegit
]

def authenticate_gmail():
    """
    Autentica amb Gmail API i retorna el servei.
    
    Returns:
        Gmail API service object
    """
    creds = None
    
    # Si ja existeix token.json, el carrega
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    
    # Si no hi ha credencials vàlides, autentica
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            print("🔄 Renovant token...")
            creds.refresh(Request())
        else:
            if not os.path.exists('credentials.json'):
                raise FileNotFoundError(
                    "❌ No s'ha trobat credentials.json!\n"
                    "Segueix les instruccions de la secció 5.1 per crear-lo."
                )
            
            print("🔐 Primera autenticació...")
            print("   S'obrirà un navegador per autenticar-te.")
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES
            )
            creds = flow.run_local_server(port=0)
        
        # Guardar credencials per següents vegades
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
        print("✅ Token guardat a token.json")
    
    # Crear servei Gmail
    service = build('gmail', 'v1', credentials=creds)
    
    # Obtenir info del compte
    profile = service.users().getProfile(userId='me').execute()
    print(f"✅ Autenticat com: {profile['emailAddress']}")
    
    return service

print("✅ Funció authenticate_gmail() definida")

### 5.4 Provar autenticació

Executem per autenticar-nos i verificar que funciona:

In [ ]:
# Autenticar (primera vegada obrirà navegador)
try:
    gmail_service = authenticate_gmail()
    print("\n✅ Gmail API configurada correctament!")
    print("Ja pots enviar i rebre emails programàticament.")
except FileNotFoundError as e:
    print(f"\n❌ Error: {e}")
    print("\n📋 Accions necessàries:")
    print("1. Segueix les instruccions de la secció 5.1")
    print("2. Descarrega credentials.json")
    print("3. Col·loca'l a la carpeta del projecte")
    print("4. Torna a executar aquesta cel·la")
except Exception as e:
    print(f"\n❌ Error inesperat: {e}")

### 5.5 Tool: Send Email

Ara creem la tool per enviar emails:

In [ ]:
from email.mime.text import MIMEText
import base64
from crewai.tools import tool

@tool("Send Email via Gmail")
def send_email_gmail(
    to_email: str,
    subject: str,
    body: str
) -> str:
    """
    Envia un email via Gmail API.
    
    Args:
        to_email: Email destinatari (ex: 'colleague@jcmtechnologies.com')
        subject: Assumpte de l'email
        body: Cos del missatge (text pla)
        
    Returns:
        Confirmació de l'enviament amb ID del missatge
    """
    try:
        # Autenticar
        service = authenticate_gmail()
        
        # Crear missatge
        message = MIMEText(body)
        message['to'] = to_email
        message['subject'] = subject
        
        # Codificar en base64
        raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode('utf-8')
        
        # Enviar
        sent_message = service.users().messages().send(
            userId='me',
            body={'raw': raw_message}
        ).execute()
        
        result = f"""
✅ EMAIL ENVIAT AMB ÈXIT!

Destinatari: {to_email}
Assumpte: {subject}
ID del missatge: {sent_message['id']}

L'email ha estat enviat correctament via Gmail.
"""
        return result
        
    except Exception as e:
        return f"❌ ERROR enviant email: {str(e)}"

print("✅ Tool 'send_email_gmail' creada!")

### 5.6 Tool: Fetch Emails

Tool per llegir emails rebuts:

In [ ]:
@tool("Fetch Emails from Gmail")
def fetch_emails_gmail(
    query: str = "is:unread",
    max_results: int = 10
) -> str:
    """
    Llegeix emails de la inbox de Gmail.
    
    Args:
        query: Cerca Gmail (ex: 'is:unread subject:RFQ', 'from:colleague@company.com')
        max_results: Màxim d'emails a retornar (default: 10)
        
    Returns:
        Llista d'emails amb: From, Subject, Date, Body
    """
    try:
        # Autenticar
        service = authenticate_gmail()
        
        # Cercar missatges
        results = service.users().messages().list(
            userId='me',
            q=query,
            maxResults=max_results
        ).execute()
        
        messages = results.get('messages', [])
        
        if not messages:
            return f"ℹ️ No s'han trobat emails que coincideixin amb: '{query}'"
        
        output = f"📧 S'han trobat {len(messages)} email(s):\n\n"
        output += "=" * 80 + "\n\n"
        
        for idx, msg in enumerate(messages, 1):
            # Obtenir detalls del missatge
            msg_data = service.users().messages().get(
                userId='me',
                id=msg['id'],
                format='full'
            ).execute()
            
            # Extreure headers
            headers = msg_data['payload']['headers']
            subject = next((h['value'] for h in headers if h['name'] == 'Subject'), 'Sin asunto')
            from_email = next((h['value'] for h in headers if h['name'] == 'From'), 'Desconocido')
            date = next((h['value'] for h in headers if h['name'] == 'Date'), 'Fecha desconocida')
            
            # Extreure cos del missatge
            body = ""
            if 'parts' in msg_data['payload']:
                for part in msg_data['payload']['parts']:
                    if part['mimeType'] == 'text/plain':
                        if 'data' in part['body']:
                            body = base64.urlsafe_b64decode(
                                part['body']['data']
                            ).decode('utf-8')
                            break
            else:
                if 'data' in msg_data['payload']['body']:
                    body = base64.urlsafe_b64decode(
                        msg_data['payload']['body']['data']
                    ).decode('utf-8')
            
            output += f"EMAIL {idx}\n"
            output += "-" * 80 + "\n"
            output += f"De: {from_email}\n"
            output += f"Assumpte: {subject}\n"
            output += f"Data: {date}\n"
            output += f"ID: {msg['id']}\n"
            output += "-" * 80 + "\n"
            output += f"COS DEL MISSATGE:\n\n{body[:1000]}"  # Limitar a 1000 caràcters
            if len(body) > 1000:
                output += "\n\n[... missatge truncat ...]\n"
            output += "\n" + "=" * 80 + "\n\n"
        
        return output
        
    except Exception as e:
        return f"❌ ERROR llegint emails: {str(e)}"

print("✅ Tool 'fetch_emails_gmail' creada!")

### 5.7 Provar les tools manualment

Abans de donar-les als agents, provem que funcionen:

In [ ]:
# OPCIONAL: Prova d'enviament (canvia l'email!)
# Descomenta i executa si vols provar

# test_result = send_email_gmail._run(
#     to_email="teu_email@gmail.com",  # ← Canvia això!
#     subject="Test CrewAI Gmail Integration",
#     body="Hola! Aquest és un email de prova enviat des de CrewAI.\n\nFunciona! 🎉"
# )
# print(test_result)

print("\n💡 Per provar l'enviament, descomenta el codi de dalt i executa.")
print("   Recorda canviar 'teu_email@gmail.com' pel teu email real!")

In [ ]:
# Provar lectura d'emails
print("📬 Llegint emails recents...\n")

emails = fetch_emails_gmail._run(
    query="newer_than:1d",  # Emails de les últimes 24h
    max_results=3
)

print(emails)

---
## 6. Dinàmica de la Formació: Enviar i Respondre

Ara ve la part interactiva! Cada assistent enviarà i rebrà un RFQ.

### 6.1 Organització dels grups

**Divisió dels 18 assistents:**

```
GRUP 1: DigiKey (6 persones)
  - Envien RFQ com a "JCM Technologies"
  - Reben RFQ i responen com a "DigiKey"
  - Format de resposta: Taula estructurada

GRUP 2: Mouser (6 persones)
  - Envien RFQ com a "JCM Technologies"
  - Reben RFQ i responen com a "Mouser"
  - Format de resposta: Text lliure, conversacional

GRUP 3: Farnell (6 persones)
  - Envien RFQ com a "JCM Technologies"
  - Reben RFQ i responen com a "Farnell"
  - Format de resposta: Caòtic/realista
```

**Flux:**
```
Persona A (Grup DigiKey) → envia RFQ a → Persona B (Grup Mouser)
Persona B (Grup Mouser) → respon com Mouser → Persona A (Grup DigiKey)
```

### 6.2 Templates de resposta per cada grup

Proporcionem templates per facilitar les respostes:

In [ ]:
# Templates de resposta per als assistents

TEMPLATE_DIGIKEY = """
De: sales@digikey.com
Assumpte: RE: Sol·licitud de cotització #RFQ-2025-001

Benvolgut/da,

Gràcies per la vostra consulta. Us adjunto la cotització sol·licitada:

Part Number          | Qty | Unit Price | Stock    | Lead Time
---------------------|-----|------------|----------|----------
IC-ATMEGA328P        | 2   | €3.20      | 500+ pcs | 2 dies
RELAY-5V-10A         | 4   | €2.50      | 200 pcs  | 5 dies
PCB-CTRL-V2.1        | 1   | €15.00     | 50 pcs   | 3 setmanes
[... afegeix més components ...]

Preus vàlids fins: 15/02/2025
MOQ: 10 unitats per component
Enviament: €25 (gratuït >€500)

Salutacions,
DigiKey Sales Team
"""

TEMPLATE_MOUSER = """
De: ventas@mouser.es
Assumpte: Cotización - Componentes JCM Technologies

Hola,

He revisado tu solicitud y te comento:

Para el microcontrolador ATMEGA328P tenemos stock disponible 
a 2,95€/ud si compras más de 25 unidades. El plazo de entrega 
es de 1 semana aproximadamente.

Los relés RELAY-5V-10A están a 2,30€ cada uno, tenemos 150 
unidades en almacén y te los podemos enviar mañana mismo.

[... afegeix més components en text lliure ...]

¿Necesitas algo más?

Saludos,
Carlos - Mouser España
"""

TEMPLATE_FARNELL = """
De: info@farnell.com
Assumpte: Re: Presupuesto

Buenos días,

ATMEGA328P -> 3.45 EUR (stock: SI, ~300pcs) 
Lead time 3-5 días laborables

RELAY 5V 10A ref RELAY-5V-10A: 2.75€ tenemos 80ud entrega inmediata!!!

[... afegeix més en format caòtic ...]

IVA no incluido
Portes: 15€ (gratis pedidos >300€)

Un saludo
"""

print("📋 Templates de resposta creats:\n")
print("1. TEMPLATE_DIGIKEY - Format taula estructurada")
print("2. TEMPLATE_MOUSER - Format text lliure conversacional")
print("3. TEMPLATE_FARNELL - Format caòtic realista")
print("\n💡 Cada grup usarà el seu template per respondre!")

### 6.3 Agent Email Sender (actualitzat amb Gmail)

Actualitzem l'agent RFQ Generator per usar la nova tool de Gmail:

In [ ]:
from crewai import Agent

agent_email_sender = Agent(
    role="Especialista en Enviar RFQs per Email",
    goal="Enviar emails RFQ professionals a companys de treball simulant proveïdors",
    backstory="""
    Ets un especialista en comunicació B2B amb 10 anys d'experiència.
    
    La teva expertesa és:
    - Preparar el contingut d'emails RFQ
    - Enviar emails via Gmail de manera professional
    - Assegurar que tota la informació necessària està present
    - Confirmar l'enviament correcte
    
    Tens accés a eines per enviar emails via Gmail API.
    
    Sempre confirmes que l'email s'ha enviat correctament abans de continuar.
    """,
    tools=[send_email_gmail],
    llm="gemini/gemini-2.5-flash",
    verbose=True
)

print("✅ Agent 'Email Sender' creat amb Gmail tool!")

### 6.4 Configuració de parelles

**⚠️ IMPORTANT:** Abans d'executar, cada assistent ha de configurar:

In [ ]:
# ═══════════════════════════════════════════════════════════
# CONFIGURACIÓ PERSONAL
# ═══════════════════════════════════════════════════════════
# Cada assistent ha d'omplir això amb el seu email i el del company assignat:

MY_EMAIL = "teu_nom@jcmtechnologies.com"  # ← El teu email
PARTNER_EMAIL = "company@jcmtechnologies.com"  # ← Email del company assignat

# El teu rol (el proveïdor que simules quan respons):
MY_SUPPLIER_ROLE = "DigiKey"  # ← Canvia per: "DigiKey", "Mouser" o "Farnell"

# ═══════════════════════════════════════════════════════════

print(f"✅ Configuració:")
print(f"   El teu email: {MY_EMAIL}")
print(f"   Enviaràs RFQ a: {PARTNER_EMAIL}")
print(f"   Quan responguis, seràs: {MY_SUPPLIER_ROLE}")

### 6.5 Task: Enviar RFQ al company

Aquesta task envia l'RFQ generat anteriorment al company assignat:

In [ ]:
from crewai import Task

task_send_rfq = Task(
    description=f"""
    Envia l'RFQ generat anteriorment al company assignat.
    
    Basant-te en els RFQs generats a la task anterior (context):
    
    1. Extreu el contingut de l'RFQ apropiat
    2. Envia'l a: {PARTNER_EMAIL}
    3. Assumpte: "Sol·licitud de cotització #RFQ-2025-001 - Components electrònics"
    
    Usa la tool 'Send Email via Gmail' per enviar-lo.
    
    Confirma que l'email s'ha enviat correctament.
    """,
    expected_output=f"""
    Confirmació de l'enviament:
    
    ✅ Email enviat a: {PARTNER_EMAIL}
    ✅ Assumpte: Sol·licitud de cotització #RFQ-2025-001
    ✅ ID del missatge: [ID de Gmail]
    
    L'email ha estat enviat correctament i el company hauria de rebre'l en uns segons.
    """,
    agent=agent_email_sender,
    context=[task_generate_rfqs]  # Usa els RFQs generats abans
)

print("✅ Task 'Send RFQ' creada!")
print(f"   Enviarà a: {PARTNER_EMAIL}")

### 6.6 Executar: Enviar RFQ

Ara executem el sistema complet fins aquest punt:

In [ ]:
from crewai import Crew

# Crew que analitza BOM, genera RFQs i els envia
crew_send_rfq = Crew(
    agents=[agent_bom_analyzer, agent_rfq_generator, agent_email_sender],
    tasks=[task_analyze_bom, task_generate_rfqs, task_send_rfq],
    verbose=True
)

print("🚀 Executant sistema complet: Analitzar BOM → Generar RFQ → Enviar Email\n")
print("=" * 70)
print(f"\n📧 S'enviarà un RFQ a: {PARTNER_EMAIL}")
print("\n⏳ Això pot trigar 1-2 minuts...\n")
print("=" * 70)

result_send = crew_send_rfq.kickoff()

print("\n" + "=" * 70)
print("✅ Procés completat!")
print("=" * 70)
print("\n📊 RESULTAT:\n")
print(result_send)
print("\n" + "=" * 70)
print(f"\n💡 El teu company ({PARTNER_EMAIL}) hauria de rebre l'email ara.")
print("   Comprova la teva inbox també per si has rebut un RFQ d'algú altre!")

### 6.7 BREAK: Respondre als RFQs (10-15 minuts)

**🕐 Ara farem un BREAK de 10-15 minuts perquè:**

1. ✅ Cada assistent comprovi si ha rebut un RFQ
2. ✅ Respongui al RFQ simulant el seu proveïdor assignat
3. ✅ Usi el template corresponent (DigiKey/Mouser/Farnell)
4. ☕ Aprofitar per un cafè!

**📋 Instruccions per respondre:**

1. Obre el teu Gmail
2. Busca l'email amb assumpte "Sol·licitud de cotització #RFQ-2025-001"
3. Respon usant el template del teu grup:
   - **Grup DigiKey:** Format taula estructurada
   - **Grup Mouser:** Text lliure conversacional
   - **Grup Farnell:** Format caòtic realista
4. **Inventa preus realistes** pels components (no cal que siguin reals)
5. Envia la resposta!

**💡 Consells:**
- Posa preus variats (alguns cars, alguns econòmics)
- Varia stocks (alguns "en stock", alguns "3-4 setmanes")
- Fes-ho divertit! Simula el teu proveïdor

In [ ]:
# Mostra el template corresponent segons el rol
print(f"📋 EL TEU TEMPLATE ({MY_SUPPLIER_ROLE}):\n")
print("=" * 80)

if MY_SUPPLIER_ROLE == "DigiKey":
    print(TEMPLATE_DIGIKEY)
elif MY_SUPPLIER_ROLE == "Mouser":
    print(TEMPLATE_MOUSER)
elif MY_SUPPLIER_ROLE == "Farnell":
    print(TEMPLATE_FARNELL)
else:
    print("⚠️ Rol no reconegut! Tria: DigiKey, Mouser o Farnell")

print("\n" + "=" * 80)
print("\n💡 Copia aquest template, omple'l amb preus i respon l'email!")

---
## 7. Agent 3: Email Parser ⭐

**Aquesta és la part més interessant!** L'agent que parseja emails reals amb formats diversos.

### 7.1 Per què aquesta és la part clau?

**El desafiament:**
- ❌ Els emails NO estan estructurats
- ❌ Cada proveïdor usa un format diferent
- ❌ Pot faltar informació
- ❌ Poden haver errors tipogràfics
- ❌ Pot estar en idiomes diversos

**El poder dels LLMs:**
- ✅ Entenen context i significat
- ✅ Poden extreure info de text no estructurat
- ✅ Gestionen formats diversos
- ✅ Normalitzen dades automàticament
- ✅ Toleren errors i inconsistències

**Això NO es pot fer amb regex o parsers tradicionals!**

### 7.2 Crear l'Agent Email Parser

In [ ]:
agent_email_parser = Agent(
    role="Expert en Parsing d'Emails de Cotitzacions",
    goal="Extreure preus, stock i condicions d'emails de proveïdors amb formats diversos",
    backstory="""
    Ets un especialista en processament de llenguatge natural amb 10 anys d'experiència
    processant emails comercials.
    
    La teva expertesa és:
    - Llegir emails de proveïdors en formats MOLT diversos
    - Extreure informació clau: preus, stock, lead times, condicions
    - Normalitzar dades (convertir monedes, unitats, dates)
    - Gestionar informació incompleta o ambigua
    - Detectar warnings o condicions especials
    
    Pots processar:
    - Taules ben formatades
    - Text lliure conversacional
    - Formats caòtics o inconsistents
    - Múltiples idiomes (català, castellà, anglès)
    
    Tens accés a eines per llegir emails de Gmail.
    
    MOLT IMPORTANT:
    - Si no trobes una dada, marca-la com "No especificat" (NO inventes!)
    - Si hi ha ambigüitat, indica-ho a les notes
    - Converteix sempre a EUR si cal
    - Normalitza lead times a format estàndard (ex: "3-5 dies", "2 setmanes")
    
    El teu output ha de ser structured amb Pydantic per poder-lo processar després.
    """,
    tools=[fetch_emails_gmail],
    llm="gemini/gemini-2.5-flash",
    verbose=True
)

print("✅ Agent 'Email Parser' creat!")

### 7.3 Task: Parse Email Response

In [ ]:
task_parse_email = Task(
    description=f"""
    Llegeix la resposta al RFQ que has rebut i extreu tota la informació de cotització.
    
    Passos:
    1. Usa la tool 'Fetch Emails from Gmail' per llegir emails recents
       Cerca: "subject:cotització OR subject:RFQ newer_than:1d"
    
    2. Identifica l'email de resposta del teu company ({PARTNER_EMAIL})
    
    3. Per CADA component mencionat a l'email, extreu:
       - Part Number (exacte)
       - Preu unitari (convertir a EUR si cal)
       - Stock disponible (número o "En stock" / "Sense stock")
       - Lead time (temps de lliurament)
       - MOQ (Minimum Order Quantity) si està especificat
       - Qualsevol nota especial
    
    4. Extreu també informació general:
       - Cost d'enviament
       - Condicions de pagament
       - Validesa de la cotització
    
    CRÍTIC:
    - Gestiona formats diversos (taula, text lliure, caòtic)
    - Si una dada no està, marca-la com "No especificat" (NO inventes!)
    - Normalitza preus (sempre en EUR)
    - Normalitza lead times (format estàndard)
    """,
    expected_output="""
    Dades extretes de l'email en format estructurat:
    
    ## Informació del Proveïdor
    - Nom: [nom del proveïdor]
    - Email: [email]
    - Data resposta: [data]
    
    ## Components Cotitzats
    
    ### Component 1
    - Part Number: [codi]
    - Preu unitari: €X.XX
    - Stock: [número] unitats / "En stock" / "Sense stock"
    - Lead time: [temps]
    - MOQ: [número] (si aplica)
    - Notes: [notes especials]
    
    ### Component 2
    [... repetir per cada component ...]
    
    ## Condicions Generals
    - Cost enviament: €XX o condicions
    - Pagament: [terminis]
    - Validesa: [data límit]
    - Notes addicionals: [si n'hi ha]
    """,
    agent=agent_email_parser,
    output_pydantic=SupplierResponse  # ← Structured output!
)

print("✅ Task 'Parse Email' creada amb structured output (Pydantic)!")

### 7.4 Executar Email Parser

**⚠️ Assegura't que:**
1. Has enviat el teu RFQ (secció 6.6)
2. El teu company t'ha respost
3. Ha passat almenys 1-2 minuts perquè arribi l'email

In [ ]:
# Crew temporal per provar el parser
crew_parse = Crew(
    agents=[agent_email_parser],
    tasks=[task_parse_email],
    verbose=True
)

print("🚀 Executant Email Parser...\n")
print("=" * 70)
print(f"\n📬 Buscant resposta de: {PARTNER_EMAIL}")
print("\n⏳ Llegint i processant email...\n")
print("=" * 70)

result_parse = crew_parse.kickoff()

print("\n" + "=" * 70)
print("✅ Parsing completat!")
print("=" * 70)
print("\n📊 DADES EXTRETES:\n")
print(result_parse)

# Si és un objecte Pydantic, mostrar-lo de forma elegant
if hasattr(result_parse, 'pydantic'):
    parsed_data = result_parse.pydantic
    print("\n" + "=" * 70)
    print("📋 DADES ESTRUCTURADES (Pydantic):\n")
    print(f"Proveïdor: {parsed_data.supplier}")
    print(f"Data: {parsed_data.response_date}")
    print(f"\nComponents cotitzats: {len(parsed_data.quotes)}")
    for quote in parsed_data.quotes:
        print(f"\n  - {quote.part_number}")
        print(f"    Preu: €{quote.unit_price}")
        print(f"    Stock: {quote.quantity_available or 'No especificat'}")
        print(f"    Lead time: {quote.lead_time}")

### 💡 Observa la màgia!

**El que ha passat:**
1. ✅ L'agent ha llegit un email REAL
2. ✅ Ha entès el format (taula/text/caòtic)
3. ✅ Ha extret preus, stocks, lead times
4. ✅ Ha normalitzat les dades
5. ✅ Ha generat un objecte Pydantic estructurat

**Sense LLMs això seria:**
- ❌ Regex complexos per cada format
- ❌ Múltiples parsers específics
- ❌ Gestió manual d'excepcions
- ❌ Manteniment constant quan formats canvien

**Amb LLMs:**
- ✅ Un sol agent
- ✅ Gestiona tots els formats
- ✅ S'adapta automàticament
- ✅ Entén context i significat

---
## 8. Agents 4 i 5: Comparació i Informe

Ara que tenim les dades extretes, fem la comparativa i generem l'informe final.

### 8.1 Agent Price Comparator

In [ ]:
agent_price_comparator = Agent(
    role="Expert en Comparació de Preus i Condicions",
    goal="Comparar cotitzacions de múltiples proveïdors i identificar les millors ofertes",
    backstory="""
    Ets un analista de procurement amb 15 anys d'experiència a JCM Technologies.
    
    La teva expertesa és:
    - Comparar preus de múltiples proveïdors
    - Avaluar NO només el preu, sinó també:
      * Disponibilitat de stock
      * Lead times
      * Condicions de pagament
      * Costos d'enviament
      * Fiabilitat del proveïdor
    
    - Identificar la millor oferta per cada component
    - Calcular costos totals (preu + enviament)
    - Detectar alertes:
      * Stock baix
      * Lead times llargs (>2 setmanes)
      * Preus anormalment alts o baixos
      * MOQ problemàtics
    
    - Generar recomanacions pràctiques
    
    El teu anàlisi sempre considera el COST TOTAL, no només el preu unitari.
    
    Per exemple:
    - Component €3.00 + enviament gratis > Component €2.50 + enviament €30
    - Stock immediat > Preu 10% més baix amb 6 setmanes lead time
    """,
    tools=[],  # No necessita tools, treballa amb dades de context
    llm="gemini/gemini-2.5-flash",
    verbose=True
)

print("✅ Agent 'Price Comparator' creat!")

### 8.2 Task: Compare Prices

In [ ]:
task_compare_prices = Task(
    description="""
    Analitza les cotitzacions extretes dels emails i genera una comparativa completa.
    
    Basant-te en les dades de la task anterior (context):
    
    1. Per CADA component:
       - Compara preus unitaris
       - Avalua disponibilitat de stock
       - Compara lead times
       - Identifica la millor oferta
       - Calcula cost total: (preu unitari × quantitat) + proporció enviament
    
    2. Detecta alertes:
       - Components amb stock <10 unitats
       - Lead times >3 setmanes
       - Preus >50% més cars que la mitjana
       - MOQ superior a la quantitat necessària
    
    3. Genera recomanacions:
       - Millor proveïdor per cada component
       - Possibles estalvis
       - Riscos a considerar
       - Alternativa si millor oferta no disponible
    
    4. Calcula cost total del projecte amb:
       - Millor opció per cada component
       - Pitjor opció (per comparar estalvi)
    """,
    expected_output="""
    Informe de comparativa amb aquesta estructura:
    
    ## Resum Executiu
    - Cost total amb millors ofertes: €XXX.XX
    - Cost total amb pitjors ofertes: €XXX.XX
    - Estalvi potencial: €XX.XX (XX%)
    - Components analitzats: [número]
    - Alertes detectades: [número]
    
    ## Comparativa per Component
    
    ### [Part Number]
    - Quantitat necessària: X unitats
    - Proveïdors comparats: [llista]
    - Millor oferta: [Proveïdor] - €X.XX/ud (total: €XX.XX)
    - Diferència vs pitjor: €X.XX (XX%)
    - Stock: [info]
    - Lead time: [temps]
    - ⚠️ Alertes: [si n'hi ha]
    - 💡 Recomanació: [text]
    
    [... repetir per cada component ...]
    
    ## Recomanacions Finals
    1. [Recomanació prioritària]
    2. [Recomanació secundària]
    3. [Consideracions addicionals]
    """,
    agent=agent_price_comparator,
    context=[task_parse_email]  # Rep les dades parsejades
)

print("✅ Task 'Compare Prices' creada!")

### 8.3 Agent Report Generator

In [ ]:
agent_report_generator = Agent(
    role="Especialista en Informes Executius",
    goal="Generar informes professionals i clars per la direcció",
    backstory="""
    Ets un expert en comunicació corporativa amb 10 anys d'experiència.
    
    La teva especialitat és:
    - Generar informes executius clars i concisos
    - Presentar dades complexes de manera entenedora
    - Destacar insights clau (no només dades)
    - Usar visualitzacions quan cal
    - Adaptar el to segons l'audiència
    
    Per informes a direcció:
    - ✅ Comença amb el més important (resum executiu)
    - ✅ Usa números i percentatges (ROI, estalvis)
    - ✅ Destaca riscos i oportunitats
    - ✅ Proporciona recomanacions accionables
    - ✅ Format professional
    
    Els teus informes sempre responen:
    - Què hem trobat?
    - Quant ens costa?
    - Què recomanem?
    - Quins riscos hi ha?
    """,
    tools=[],
    llm="gemini/gemini-2.5-flash",
    verbose=True
)

print("✅ Agent 'Report Generator' creat!")

### 8.4 Task: Generate Final Report

In [ ]:
task_generate_report = Task(
    description="""
    Genera un informe executiu professional amb tota la informació de cotització.
    
    Basant-te en l'anàlisi comparativa anterior (context):
    
    1. Resum Executiu (el més important al principi):
       - Cost total recomanat
       - Estalvi vs alternatives
       - Principal recomanació
       - Riscos crítics
    
    2. Detall de Cotització:
       - Taula comparativa de components
       - Millor proveïdor per cada component
       - Costos desglosats
    
    3. Anàlisi de Riscos:
       - Components amb stock baix
       - Lead times llargs
       - Dependències d'un sol proveïdor
    
    4. Recomanacions:
       - Ordre de compra recomanat
       - Alternatives per components crítics
       - Accions de seguiment
    
    FORMAT: Markdown professional amb taules i seccions clares.
    TO: Executiu però tècnic (per R+D)
    LONGITUD: Complet però concís (2-3 pàgines)
    """,
    expected_output="""
    Un informe complet en format Markdown amb:
    
    # Informe de Cotització - [Projecte]
    *Data: [data]*
    *Generat per: Sistema automatitzat CrewAI*
    
    ## 📊 Resum Executiu
    [Paràgraf amb el més important]
    
    **Mètriques Clau:**
    - Cost total: €XXX.XX
    - Estalvi vs alternativa: €XX.XX (XX%)
    - Components cotitzats: XX
    - Proveïdors analitzats: X
    
    ## 💰 Detall de Cotització
    [Taula comparativa]
    
    ## ⚠️ Anàlisi de Riscos
    [Llista de riscos]
    
    ## 💡 Recomanacions
    [Llista de recomanacions accionables]
    
    ## 📝 Accions de Seguiment
    [Què cal fer ara]
    """,
    agent=agent_report_generator,
    context=[task_compare_prices]
)

print("✅ Task 'Generate Report' creada!")

---
## 9. Integració Completa del Sistema

Ara executem TOT el sistema de cap a cap!

### 9.1 Crear el Crew complet amb tots els agents

In [ ]:
# Crew complet amb tots els agents i tasks
crew_complete_system = Crew(
    agents=[
        agent_bom_analyzer,
        agent_rfq_generator,
        agent_email_sender,
        # (Break: Companys responen)
        agent_email_parser,
        agent_price_comparator,
        agent_report_generator
    ],
    tasks=[
        task_analyze_bom,          # 1. Analitzar BOM
        task_generate_rfqs,        # 2. Generar RFQs
        task_send_rfq,             # 3. Enviar per email
        # [BREAK aquí per respostes]
        task_parse_email,          # 4. Parsear resposta
        task_compare_prices,       # 5. Comparar preus
        task_generate_report       # 6. Generar informe
    ],
    process=Process.sequential,
    verbose=True
)

print("✅ Crew complet creat amb 6 agents i 6 tasks!")
print("\n📊 Workflow:")
print("  1. BOM Analyzer → Identifica components")
print("  2. RFQ Generator → Genera emails")
print("  3. Email Sender → Envia via Gmail")
print("  4. [BREAK] → Companys responen")
print("  5. Email Parser → Extreu dades")
print("  6. Price Comparator → Analitza ofertes")
print("  7. Report Generator → Informe final")

### 9.2 Opció A: Executar tot (incloent l'enviament)

**⚠️ NOTA:** Això enviarà un email REAL. Només executa si vols fer tot el procés complet.

In [ ]:
# DESCOMENTAR per executar el sistema complet:

# print("🚀 EXECUTANT SISTEMA COMPLET DE COTITZACIÓ\n")
# print("=" * 70)
# print("\n⚠️ Aquest procés:")
# print("  1. Analitzarà el BOM")
# print("  2. Generarà RFQs")
# print(f"  3. Enviarà email a {PARTNER_EMAIL}")
# print("  4. Esperarà que responguis")
# print("  5. Parsearà la resposta")
# print("  6. Compararà preus")
# print("  7. Generarà informe final")
# print("\n⏳ Trigarà ~5 minuts (sense comptar el temps de resposta)\n")
# print("=" * 70)

# result_complete = crew_complete_system.kickoff()

# print("\n" + "=" * 70)
# print("✅ SISTEMA COMPLET EXECUTAT!")
# print("=" * 70)
# print("\n📊 INFORME FINAL:\n")
# print(result_complete)

print("💡 Descomenta el codi de dalt per executar el sistema complet.")
print("   O continua amb l'Opció B per executar només des del parsing.")

### 9.3 Opció B: Executar només parsing + comparació + informe

Si ja has enviat l'RFQ manualment i tens respostes, usa això:

In [ ]:
# Crew només amb parsing, comparació i informe
crew_parse_to_report = Crew(
    agents=[agent_email_parser, agent_price_comparator, agent_report_generator],
    tasks=[task_parse_email, task_compare_prices, task_generate_report],
    process=Process.sequential,
    verbose=True
)

print("🚀 Executant des del parsing fins l'informe...\n")
print("=" * 70)
print(f"\n📬 Buscant resposta de: {PARTNER_EMAIL}")
print("\n⏳ Processant...\n")
print("=" * 70)

result_parse_to_report = crew_parse_to_report.kickoff()

print("\n" + "=" * 70)
print("✅ PROCÉS COMPLETAT!")
print("=" * 70)
print("\n📊 INFORME FINAL:\n")
print(result_parse_to_report)

### 9.4 Guardar l'informe

Guardem l'informe generat a un fitxer Markdown:

In [ ]:
import os
from datetime import datetime

# Crear directori d'outputs si no existeix
os.makedirs('outputs', exist_ok=True)

# Nom del fitxer amb timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"outputs/informe_cotitzacio_{timestamp}.md"

# Guardar informe
with open(filename, 'w', encoding='utf-8') as f:
    f.write(str(result_parse_to_report))

print(f"✅ Informe guardat a: {filename}")
print(f"\n💡 Pots obrir-lo amb qualsevol editor de Markdown o convertir-lo a PDF.")

---
## 10. Bonus: Millores i Properes Passes

Has completat el sistema bàsic! Ara vegem com millorar-lo.

### 10.1 Millores immediates

**1. Gestió d'errors més robusta:**

```python
try:
    result = crew.kickoff()
except Exception as e:
    print(f"Error: {e}")
    # Intentar amb mock data
    # O alertar l'usuari
    # O usar fallback strategy
```

**2. Múltiples respostes per RFQ:**

En lloc d'enviar a 1 company, envia a 3:
- Parseja les 3 respostes
- Compara entre totes
- Millor anàlisi

**3. Base de dades de cotitzacions:**

```python
import sqlite3
# Guardar totes les cotitzacions històriques
# Analitzar tendències de preus
# Detectar proveïdors fiables
```

**4. Notificacions automàtiques:**

- Slack: Alertar quan s'ha rebut resposta
- Email: Enviar informe a la direcció
- Dashboard: Visualització en temps real

### 10.2 Integrar amb sistemes reals

**ERP Integration:**

```python
# Llegir BOMs directament de l'ERP
# Crear ordres de compra automàticament
# Actualitzar inventari
```

**Outlook en lloc de Gmail:**

```python
from msal import ConfidentialClientApplication
import requests

# Microsoft Graph API
# Similar a Gmail però per Microsoft 365
```

**APIs reals de proveïdors:**

Si el proveïdor té API (Digi-Key, Mouser, Octopart):
- Consulta directa de preus
- Stock en temps real
- Més ràpid que email

### 10.3 Escalabilitat

**Per processar múltiples BOMs:**

```python
# Processar en paral·lel
from concurrent.futures import ThreadPoolExecutor

boms = ['bom1.xlsx', 'bom2.xlsx', 'bom3.xlsx']

with ThreadPoolExecutor(max_workers=3) as executor:
    results = executor.map(process_bom, boms)
```

**Hierarchical Process per BOMs complexos:**

```python
crew = Crew(
    agents=[manager, worker1, worker2, worker3],
    tasks=[complex_task],
    process=Process.hierarchical,
    manager_llm="gemini/gemini-2.5-flash"
)
# El manager distribueix subtasques automàticament
```

### 10.4 Recursos per continuar aprenent

**Documentació oficial:**
- CrewAI: https://docs.crewai.com/
- Gmail API: https://developers.google.com/gmail/api
- Pydantic: https://docs.pydantic.dev/

**Cursos recomanats:**
- DeepLearning.AI: "AI Agents in LangGraph"
- CrewAI official tutorials
- Google Cloud Skills Boost: Gmail API

**Comunitats:**
- CrewAI Discord: https://discord.gg/crewai
- r/LangChain (Reddit)
- AI Engineers community

**Projectes similars per inspirar-se:**
- Automatització de factures
- Processament de CVs
- Anàlisi de feedback de clients
- Generació de propostes comercials

---
## 🎉 Felicitats!

Has completat el cas pràctic complet i has construït un sistema real i funcional!

**El que has après:**
- ✅ Crear sistemes multi-agent complexos
- ✅ Integrar amb Gmail API (email real)
- ✅ Parsing intel·ligent de text no estructurat
- ✅ Structured outputs amb Pydantic
- ✅ Workflows seqüencials amb dependències
- ✅ Gestió d'errors i fallbacks
- ✅ Generació d'informes professionals

**El que has construït:**
Un sistema que:
1. Analitza BOMs automàticament
2. Genera i envia RFQs per email
3. Parseja respostes en formats diversos
4. Compara preus i condicions
5. Genera informes executius

**I tot això amb només 6 agents i ~300 línies de codi! 🚀**

**Properes passes:**
- Adapta'l al teu cas d'ús específic a JCM
- Integra amb els vostres sistemes (ERP, Outlook, etc.)
- Afegeix més agents per funcionalitats addicionals
- Comparteix amb l'equip i itereu junts

**Gràcies per participar en aquesta formació! 🙏**